**Spark SQL trabaja con DataFrames**. Un DataFrame, como ya lo hemos comentado es una **representación relacional de los datos**. Proporciona funciones con capacidades similares a SQL. Además, permite escribir **consultas tipo SQL** para nuestro análisis de datos.

### Creacion de un df desde 0

In [0]:
emp = [(1, "AAAAA", "dept1", 1000),
    (2, "BBBBB", "dept1", 1100),
    (3, "CCCCC", "dept1", 2000),
    (4, "DDDDD", "dept1", 3500),
    (5, "EEEEE", "dept2", 8000),
    (6, "FFFFF", "dept2", 5200),
    (7, "GGGGG", "dept3", 3100),
    (8, "HHHHH", "dept3", 6700),
    (9, "IIIII", "dept3", 6500),
    (10, "JJJJJ", "dept4", 5400)]

dept = [("dept1", "Department - 1"),
        ("dept2", "Department - 2"),
        ("dept3", "Department - 3"),
        ("dept4", "Department - 4")
       ]

dfemp = spark.createDataFrame(emp, ["id", "name", "dept", "salary"])

deptdf = spark.createDataFrame(dept, ["id", "name"])

In [0]:
dfemp.show()

+---+-----+-----+------+
| id| name| dept|salary|
+---+-----+-----+------+
|  1|AAAAA|dept1|  1000|
|  2|BBBBB|dept1|  1100|
|  3|CCCCC|dept1|  2000|
|  4|DDDDD|dept1|  3500|
|  5|EEEEE|dept2|  8000|
|  6|FFFFF|dept2|  5200|
|  7|GGGGG|dept3|  3100|
|  8|HHHHH|dept3|  6700|
|  9|IIIII|dept3|  6500|
| 10|JJJJJ|dept4|  5400|
+---+-----+-----+------+



In [0]:
deptdf.show()

# Operaciones básicas en DataFrames

Podemos aplicar las transformaciones que ya hemos visto en la seccion de RDDs, por ejemplo:

In [0]:
dfemp.count()

In [0]:
dfemp.printSchema()

In [0]:
dfemp.select("id", "name").show()

+---+-----+
| id| name|
+---+-----+
|  1|AAAAA|
|  2|BBBBB|
|  3|CCCCC|
|  4|DDDDD|
|  5|EEEEE|
|  6|FFFFF|
|  7|GGGGG|
|  8|HHHHH|
|  9|IIIII|
| 10|JJJJJ|
+---+-----+



### Ejemplo avanzado de: filter

* Filtrar las filas según alguna condición.
* Intentemos encontrar las filas con id = 1.
* Hay diferentes formas de especificar la condición.

In [0]:
dfemp.filter(dfemp["id"] == 1).show()
dfemp.filter(dfemp.id == 1).show()

In [0]:
from pyspark.sql.functions import col

dfemp.filter(col("id") == 1).show()
dfemp.filter("id = 1").show()

+---+-----+-----+------+
| id| name| dept|salary|
+---+-----+-----+------+
|  1|AAAAA|dept1|  1000|
+---+-----+-----+------+

+---+-----+-----+------+
| id| name| dept|salary|
+---+-----+-----+------+
|  1|AAAAA|dept1|  1000|
+---+-----+-----+------+



### Funcion: drop
* Elimina una columna en particular

In [0]:
newdf = dfemp.drop("id")
newdf.show(2)

### Funcion: withColumn
* Podemos usar la función "withColumn" para derivar la columna en función de las columnas existentes.

In [0]:
dfemp.withColumn("bonus", col("salary") * .1).show()

+---+-----+-----+------+-----+
| id| name| dept|salary|bonus|
+---+-----+-----+------+-----+
|  1|AAAAA|dept1|  1000|100.0|
|  2|BBBBB|dept1|  1100|110.0|
|  3|CCCCC|dept1|  2000|200.0|
|  4|DDDDD|dept1|  3500|350.0|
|  5|EEEEE|dept2|  8000|800.0|
|  6|FFFFF|dept2|  5200|520.0|
|  7|GGGGG|dept3|  3100|310.0|
|  8|HHHHH|dept3|  6700|670.0|
|  9|IIIII|dept3|  6500|650.0|
| 10|JJJJJ|dept4|  5400|540.0|
+---+-----+-----+------+-----+



### Ejemplo de agregacion:
* Podemos usar la función groupBy para agrupar los datos y luego usar la función "agg" para realizar la agregación de datos agrupados.

In [0]:
from pyspark.sql import functions as f
withColumn
(dfemp.groupBy("dept")
    .agg(
        f.count("salary").alias("conteo"),
        f.sum("salary").alias("suma"),
        f.max("salary").alias("maximo"),
        f.min("salary").alias("minimo"),
        f.avg("salary").alias("promedio"))
    .show()
)

+-----+------+-----+------+------+-----------------+
| dept|conteo| suma|maximo|minimo|         promedio|
+-----+------+-----+------+------+-----------------+
|dept1|     4| 7600|  3500|  1000|           1900.0|
|dept2|     2|13200|  8000|  5200|           6600.0|
|dept3|     3|16300|  6700|  3100|5433.333333333333|
|dept4|     1| 5400|  5400|  5400|           5400.0|
+-----+------+-----+------+------+-----------------+



### Por ultimo, tambien podemos hacer joins, como en SQL

In [0]:
# Inner JOIN.
dfemp.join(deptdf, dfemp["dept"] == deptdf["id"]).show()

### Left Outer Join

In [0]:
dfemp.join(deptdf, dfemp["dept"] == deptdf["id"], "left_outer").show()

### Right Outer Join

In [0]:
dfemp.join(deptdf, dfemp["dept"] == deptdf["id"], "right_outer").show()

### Full Outer Join

In [0]:
dfemp.join(deptdf, dfemp["dept"] == deptdf["id"], "outer").show()

In [0]:
#Desde la librería "pyspark.sql.types" importamos los utilitarios "StructType" y el "StructField"
#"StrucType" nos permite modificar el esquema de metadatos de un dataframe
#"StructField" nos permite modificar a un campo del esquema de metadatos.
#Tambien es necesario importar los tipos de datos que utilizaremos
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [0]:
# Leemos el archivo de persona , con un esquema de metadatos predefinido

dfPersona = spark.read.format("csv").option("header","true").option("delimiter","|").schema(
    StructType(
        [
            StructField("ID",StringType(),True),
            StructField("NOMBRE",StringType(),True),
            StructField("TELEFONO", StringType(), True),
            StructField("CORREO", StringType(), True),
            StructField("FECHA_INGRESO", StringType(), True),
            StructField("EDAD", IntegerType(), True),
            StructField("SALARIO", DoubleType(), True),
            StructField("ID_EMPRESA", StringType(), True)

        ]
    )
).load("dbfs:/FileStore/shared_uploads/rcortezrosas@gmail.com/persona.data")

In [0]:
#Mostramos los datos
dfPersona.show(5)

+---+---------+--------------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|
|  3|  Jocelyn|1-204-956-8594|amet.diam@loborti...|   2002-08-01|  27|10853.0|         3|
|  4|    Aidan|1-719-862-9385|euismod.et.commod...|   2018-11-06|  29| 3387.0|        10|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
only showing top 5 rows



In [0]:
#Creacion de la vista temporal para utilizar con Spark SQL
dfPersona.createOrReplaceTempView('dfPersona')

In [0]:
# Filtramos alfunos regitros segun la edad
spark.sql("SELECT * FROM dfPersona WHERE EDAD >30").show(10)

+---+---------+--------------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|
|  6|     Bert|      797-4453|a.felis.ullamcorp...|   2017-04-25|  70| 7800.0|         7|
|  7|     Mark|1-680-102-6792|Quisque.ac@placer...|   2006-04-21|  52| 8112.0|         5|
|  9|    Hanae|      935-2277|          eu@Nunc.ca|   2003-05-25|  69| 6834.0|         3|
| 11|  Melyssa|      596-7736|vel@vulputateposu...|   2008-10-14|  48| 4913.0|         8|
| 13|   Trevor|      512-1955|Nunc.quis.arcu@eg...|   2010-08-06|  34| 9501.0|         5|
| 14|    A

In [0]:
# Usmos triple comilla para hacer comandos SQL con saltos de linea
df1 = spark.sql("""
                SELECT *
                FROM
                dfPersona
                WHERE
                EDAD > 30 """)
df1.show(10)

+---+---------+--------------+--------------------+-------------+----+-------+----------+
| ID|   NOMBRE|      TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+--------------------+-------------+----+-------+----------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|  32|20095.0|         5|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|   2019-02-17|  34| 9298.0|         2|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|   2002-10-10|  41|22102.0|         1|
|  6|     Bert|      797-4453|a.felis.ullamcorp...|   2017-04-25|  70| 7800.0|         7|
|  7|     Mark|1-680-102-6792|Quisque.ac@placer...|   2006-04-21|  52| 8112.0|         5|
|  9|    Hanae|      935-2277|          eu@Nunc.ca|   2003-05-25|  69| 6834.0|         3|
| 11|  Melyssa|      596-7736|vel@vulputateposu...|   2008-10-14|  48| 4913.0|         8|
| 13|   Trevor|      512-1955|Nunc.quis.arcu@eg...|   2010-08-06|  34| 9501.0|         5|
| 14|    A

In [0]:
# Parametrizacion de codigo SQL
PARAM_EDAD = 30
PARAM_SALARIO = 5000

In [0]:
df3 = spark.sql(f"""SELECT
                ID,NOMBRE,EDAD,SALARIO
                FROM dfPersona
                WHERE
                EDAD >{PARAM_EDAD} AND 
                SALARIO > {PARAM_SALARIO}""")
df3.show(5)

+---+---------+----+-------+
| ID|   NOMBRE|EDAD|SALARIO|
+---+---------+----+-------+
|  1|     Carl|  32|20095.0|
|  2|Priscilla|  34| 9298.0|
|  5|  Leandra|  41|22102.0|
|  6|     Bert|  70| 7800.0|
|  7|     Mark|  52| 8112.0|
+---+---------+----+-------+
only showing top 5 rows



In [0]:
#Ejercicio 1 - Datos de Prueba
# Realiar una consulta Utilizando Spark SQL para seleccionar el nombre y la edad de las personas cuya edad sea mayor a 28
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Crear una sesión de Spark
spark = SparkSession.builder.appName("ejercicio1").getOrCreate()

# Definir el esquema de los datos
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("nombre", StringType(), True),
    StructField("edad", IntegerType(), True)
])

# Crear un DataFrame de prueba
data = [(1, "Juan", 25), (2, "María", 30), (3, "Pedro", 28)]
df = spark.createDataFrame(data, schema)

# Crear una vista temporal para utilizar en Spark SQL
df.createOrReplaceTempView("personas")

In [0]:
#CON SPARK
df_1 = df.filter(df['edad']>28)
df_1.show()


+---+------+----+
| id|nombre|edad|
+---+------+----+
|  2| María|  30|
+---+------+----+



In [0]:
#Con SQL
df_2 = spark.sql(" SELECT nombre,edad FROM personas WHERE edad >28")
df_2.show()

+------+----+
|nombre|edad|
+------+----+
| María|  30|
+------+----+



In [0]:
#Ejercicio 2 - Datos de prueba
#Realiza una consulta utilizando Spark SQL para obtener la edad promedio de las personas en el DataFrame
# Utilizamos el DataFrame creado en el ejercicio 1

# Agregar más datos al DataFrame
data_nuevos = [(4, "Ana", 22), (5, "Luis", 32), (6, "Laura", 27)]
df_nuevos = spark.createDataFrame(data_nuevos, schema)

# Union de DataFrames
df = df.union(df_nuevos)
# Crear una vista temporal para utilizar en Spark SQL
df.createOrReplaceTempView("personas")

In [0]:
from pyspark.sql import functions as f
(df.agg(f.avg('edad')).show())

+------------------+
|         avg(edad)|
+------------------+
|27.333333333333332|
+------------------+



In [0]:
df_3 = spark.sql(" SELECT avg(edad) FROM personas")
df_3.show()


+------------------+
|         avg(edad)|
+------------------+
|27.333333333333332|
+------------------+



In [0]:
#Ejercicio  3 - Datos de prueba
#Realizar una consulta de SPARK SQL para obtener el nombre , la edad y la direccion de las personas que tienen una direccion registrada
# Utilizamos los DataFrames creados en los ejercicios anteriores

# Crear un segundo DataFrame para realizar un join
schema_direccion = StructType([
    StructField("id", IntegerType(), True),
    StructField("direccion", StringType(), True)
])

data_direccion = [(1, "Calle A"), (2, "Calle B"), (3, "Calle C"), (4, "Calle D")]
df_direccion = spark.createDataFrame(data_direccion, schema_direccion)

# Crear una vista temporal para el segundo DataFrame
df_direccion.createOrReplaceTempView("direcciones")

In [0]:
spark.sql(" SELECT * FROM personas").show()

+---+------+----+
| id|nombre|edad|
+---+------+----+
|  1|  Juan|  25|
|  2| María|  30|
|  3| Pedro|  28|
|  4|   Ana|  22|
|  5|  Luis|  32|
|  6| Laura|  27|
+---+------+----+



In [0]:
#con Spark

df.join(df_direccion, df.id == df_direccion.id).select("nombre", "edad", "direccion").show()

+------+----+---------+
|nombre|edad|direccion|
+------+----+---------+
|  Juan|  25|  Calle A|
| María|  30|  Calle B|
| Pedro|  28|  Calle C|
|   Ana|  22|  Calle D|
+------+----+---------+



In [0]:
spark.sql("""SELECT nombre , edad,direccion 
          FROM personas p 
          RIGHT JOIN direcciones d ON d.id == p.id""").show()

+------+----+---------+
|nombre|edad|direccion|
+------+----+---------+
|  Juan|  25|  Calle A|
| María|  30|  Calle B|
| Pedro|  28|  Calle C|
|   Ana|  22|  Calle D|
+------+----+---------+



# UDF( User Defined Functions)

¿Qué son las UDF en Apache Spark y por qué son importantes?
Dentro del mundo de Apache Spark, las UDF (User Defined Functions) juegan un rol esencial al permitir a los usuarios definir sus propias funciones personalizadas que pueden ejecutarse de manera distribuida en todos los nodos de un clúster. Esto destaca porque Spark ya ofrece funciones nativas poderosas como map, filter y flatMap. Sin embargo, el poder registrar una UDF permite a los usuarios realizar transformaciones específicas que no son posibles con las funciones nativas.

In [0]:
dfemp.show()

+---+-----+-----+------+
| id| name| dept|salary|
+---+-----+-----+------+
|  1|AAAAA|dept1|  1000|
|  2|BBBBB|dept1|  1100|
|  3|CCCCC|dept1|  2000|
|  4|DDDDD|dept1|  3500|
|  5|EEEEE|dept2|  8000|
|  6|FFFFF|dept2|  5200|
|  7|GGGGG|dept3|  3100|
|  8|HHHHH|dept3|  6700|
|  9|IIIII|dept3|  6500|
| 10|JJJJJ|dept4|  5400|
+---+-----+-----+------+



In [0]:
def detSalary_Level(salario):
    level = None

    if(salario > 5000):
        level = 'high_salary'
    elif(salario> 2000):
        level = 'mid_salary'
    elif(salario > 0):
        level = 'low_salary'
    else:
        level = 'invalid_salary'
    return level

In [0]:
#Registro de la funcion: detSalary_Level como UDF
udf_detSalary_Level = udf(detSalary_Level, StringType())

In [0]:
#Aplicacion de la udf
newdf = dfemp.withColumn("nivel_salario", udf_detSalary_Level("salary"))
newdf.show()

+---+-----+-----+------+-------------+
| id| name| dept|salary|nivel_salario|
+---+-----+-----+------+-------------+
|  1|AAAAA|dept1|  1000|   low_salary|
|  2|BBBBB|dept1|  1100|   low_salary|
|  3|CCCCC|dept1|  2000|   low_salary|
|  4|DDDDD|dept1|  3500|   mid_salary|
|  5|EEEEE|dept2|  8000|  high_salary|
|  6|FFFFF|dept2|  5200|  high_salary|
|  7|GGGGG|dept3|  3100|   mid_salary|
|  8|HHHHH|dept3|  6700|  high_salary|
|  9|IIIII|dept3|  6500|  high_salary|
| 10|JJJJJ|dept4|  5400|  high_salary|
+---+-----+-----+------+-------------+



In [0]:
#Ejercicio 1
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

def cuadrado(num):
    return num * num

cuadrado_udf = udf(cuadrado, DoubleType())#Registras la funcion en UDF

+---+-----+-----+------+
| id| name| dept|salary|
+---+-----+-----+------+
|  1|AAAAA|dept1|  1000|
|  2|BBBBB|dept1|  1100|
|  3|CCCCC|dept1|  2000|
|  4|DDDDD|dept1|  3500|
|  5|EEEEE|dept2|  8000|
|  6|FFFFF|dept2|  5200|
|  7|GGGGG|dept3|  3100|
|  8|HHHHH|dept3|  6700|
|  9|IIIII|dept3|  6500|
| 10|JJJJJ|dept4|  5400|
+---+-----+-----+------+

